load_mapping 测试

In [ ]:
import sys
import torch
import networkx as nx
import networkx.algorithms.components.connected as nxacc
import networkx.algorithms.dag as nxadag
import numpy as np
import pandas as pd
import shutil
import scipy.stats as ss # for spearman_corr

def load_mapping(mapping_file):
    """
    Opens a txt file with two columns and saves the second column as the key of the dictionary and the first column as a value.

        Parameters
        ----------
        mapping_file: str, path to txt file

        Output
        ------
        mapping: dic

        Notes: used to read gene2ind.txt, drug2ind.txt

    """
    mapping = {} # dictionary of values on required txt

    file_handle = open(mapping_file) # function opens a file, and returns it as a file object.

    for line in file_handle:
        line = line.rstrip().split() # quitar espacios al final del string y luego separar cada elemento (en gene2ind hay dos elementos 3007	ZMYND8, los pone en una lista ['3007', 'ZMYND8'] )
        mapping[line[1]] = int(line[0]) # en gene2ind el nombre del gen es el key del dictionary y el indice el valor del diccionario

    file_handle.close()

    return mapping

gene2id_mapping = load_mapping('./data/noise_free/eSOL_go/eSOL_protein2ind.txt')
print(gene2id_mapping)

{'A100048': 0, 'A100051': 1, 'A100062': 2, 'A100211': 3, 'A100223': 4, 'A100230': 5, 'A100246': 6, 'A100384': 7, 'A100496': 8, 'A100598': 9, 'A100633': 10, 'A100689': 11, 'A100702': 12, 'A101024': 13, 'A101032': 14, 'A101033': 15, 'A101046': 16, 'A101069': 17, 'A101115': 18, 'A101132': 19, 'A101348': 20, 'A101355': 21, 'A101362': 22, 'A101496': 23, 'A101775': 24, 'A101906': 25, 'A102725': 26, 'A103111': 27, 'A103121': 28, 'A103157': 29, 'A103343': 30, 'A103523': 31, 'A103549': 32, 'A103624': 33, 'A103685': 34, 'A104785': 35, 'A104969': 36, 'A105420': 37, 'A105804': 38, 'A107159': 39, 'A107816': 40, 'A108301': 41, 'A108590': 42, 'A111449': 43, 'A112027': 44, 'A112030': 45, 'A112046': 46, 'A112047': 47, 'A112051': 48, 'A112062': 49, 'A112076': 50, 'A112099': 51, 'A112100': 52, 'A112127': 53, 'A112141': 54, 'A112147': 55, 'A112153': 56, 'A112157': 57, 'A112161': 58, 'A112169': 59, 'A112170': 60, 'A112172': 61, 'A112178': 62, 'A112179': 63, 'A112185': 64, 'A112216': 65, 'A112229': 66, 'A11

load_ontology 测试

可能要减少限制  取消每个GO都要至少与一个protein有关的限制

In [ ]:

def load_ontology(ontology_file, gene2id_mapping):
    """
    Creates the directed graph of the GO terms and stores the connected elements in arrays.

        Output
        ------
        dG: networkx.classes.digraph.DiGraph
            Directed graph of all terms

        terms_pairs: numpy.ndarray
            Store the connection between a term and a term

        genes_terms_pairs: numpy.ndarray
            Store the connection between a gene and a term
    """

    dG = nx.DiGraph() # Directed graph class 初始化一个空的有向图

    file_handle = open(ontology_file) #  Open the file that has genes and go terms

    terms_pairs = [] # store the pairs between a term and a term  GO-GO列表
    genes_terms_pairs = [] # store the pairs between a gene and a term  GO-gene列表

    gene_set = set() # create a set (elements can't repeat)
    term_direct_gene_map = {}
    term_size_map = {}

    # 通过读取文件循环 就把GO-GO 的dG有向无环图创建起来了 而且也把GO-gene的dict收集起来了
    # dG - GO-GO的有向图 term_direct_gene_map - GO-gene的dict
    for line in file_handle:

        line = line.rstrip().split() # delete spaces and transform to list, line has 3 elements
        # 这三个元素分别是 GO1 GO2 default / GO1 gene1 gene 
        
        # No me hace falta el if, no tengo que separar las parejas
        if line[2] == 'default': # si el tercer elemento es default entonces se conectan los terms en el grafo
            # 如果是GO-GO  在dG有向图中添加一条边 方向就是GO1->GO2
            dG.add_edge(line[0], line[1]) # 添加边 方向GO1->GO2 父节点到子节点
            # 在GO-GO列表中添加这个pair
            terms_pairs.append([line[0], line[1]]) # Add the pair to the list
        else:
            # 如果是GO-gene 
            if line[1] not in gene2id_mapping: 
                # 并且这个gene不在gene2id_mapping中 即不在gene2ind文件中 那么打印这个gene 就跳过了
                print(line[1])
                continue
            # 到这里 就是说明 这行是GO-gene 并且gene在gene2id_mapping(gene2ind)中
            # 将这个GO-gene pair添加到GO-gene列表中
            genes_terms_pairs.append([line[0], line[1]]) # add the pair

            # term_direct_gene_map是GO为key 与该GO有关的genes为value的字典  是以GO为归纳项 (我们一般以gene为归纳项 都是说一个gene有多少GO 这里是反着的)
            # 如果这个GO不在term_direct_gene_map的Key中 那么就添加这个GO为key 并且value是一个set
            if line[0] not in term_direct_gene_map: 
                term_direct_gene_map[ line[0] ] = set() # crea un set

            # 然后这个value(GO)中添加这个gene 
            term_direct_gene_map[line[0]].add(gene2id_mapping[line[1]])

            # 将这个gene添加到gene_set中
            gene_set.add(line[1]) 

    # terms_pairs: GO-GO
    # genes_terms_pairs: GO-gene
    terms_pairs = np.array(terms_pairs) # convert to 2d array
    genes_terms_pairs = np.array(genes_terms_pairs) # convert to 2d array

    file_handle.close()

    print('There are', len(gene_set), 'genes')

    # 遍历所有的GO terms 如果该GO与所有gene都无关，则建议删掉； 否则将有关的gene存储到term_size_map中
    for term in dG.nodes(): 

        term_gene_set = set() 

        # 如果这个GO term在term_direct_gene_map中 即这个GO term有gene与之相关
        if term in term_direct_gene_map:
            # 把与这个GO有关的所有gene都加到这个set中
            term_gene_set = term_direct_gene_map[term] # genes conectados al term 

        # 将这个GO term的所有后代节点都存储到deslist中 即从该GO出发可以到达的所有节点 (在GO图中其实就是所有子节点)
        deslist = nxadag.descendants(dG, term) 

        # 这里的child表示的是当前GO的所有子节点(GO)
        for child in deslist:
            if child in term_direct_gene_map: 
                # 如果子节点也与GO有关 那么把子节点相关的gene也加到这个set中
                term_gene_set = term_gene_set | term_direct_gene_map[child] # union of both sets, ahora tiene todos los genes los suyos y los de sus descendientes
        # 上述过程 把这个GO相关的所有gene 以及这个GO的子节点的所有相关gene都存到了term_gene_set中
        # if len(term_gene_set) == 0:
        #     print('There is empty terms, please delete term:', term)
        #     sys.exit(1)
        # else:
        #     # por ahora esta variable no me hace falta
        #     # term_size_map存储的是每个GO相关的gene数量
        #     term_size_map[term] = len(term_gene_set) # cantidad de genes en ese term  (tomando en cuenta sus descendientes)
        term_size_map[term] = len(term_gene_set)


    # 找到dG中的所有根节点  即最高级的GO 祖宗级别的GO
    leaves = [n for n in dG.nodes if dG.in_degree(n) == 0] # buscar la raiz
    #leaves = [n for n,d in dG.in_degree() if d==0]

    # 有向图转无向图
    uG = dG.to_undirected() # Returns an undirected representation of the digraph
    connected_subG_list = list(nxacc.connected_components(uG)) #list of all GO terms 将无向图的连通分量存到list中
    # 即无向图中的所有子图

    # Verify my graph makes sense...
    print('There are', len(leaves), 'roots:', leaves)  # 应该只有一个根节点 就是说最后的GO图最后一层只有一个GO (这个设置可能是为了回归用的)
    print('There are', len(dG.nodes()), 'terms')
    print('There are', len(connected_subG_list), 'connected components')
    # # 如果根结点>1 会提示： 有多个根节点 请仅使用一个根节点
    # if len(leaves) > 1:
    #     print('There are more than 1 root of ontology. Please use only one root.')
    #     sys.exit(1)
    # 如果有多个子图 会提示： 有多个子图 请连通它们
    if len(connected_subG_list) > 1:
        print( 'There are more than connected components. Please connect them.')
        sys.exit(1)

    return dG, terms_pairs, genes_terms_pairs

dG, terms_pairs, genes_terms_pairs = load_ontology('./data/noise_free/eSOL_go/eSOL_total.txt', gene2id_mapping)
print(dG)
print(terms_pairs)
print(genes_terms_pairs)

There are 2846 genes
There are 1 roots: ['GO:0000000']
There are 144 terms
There are 1 connected components
DiGraph with 144 nodes and 192 edges
[['GO:0000000' 'GO:0003674']
 ['GO:0000000' 'GO:0008150']
 ['GO:0000000' 'GO:0005575']
 ['GO:0003824' 'GO:0016853']
 ['GO:0003824' 'GO:0016491']
 ['GO:0003824' 'GO:0016874']
 ['GO:0003824' 'GO:0003924']
 ['GO:0003824' 'GO:0140096']
 ['GO:0003824' 'GO:0016787']
 ['GO:0003824' 'GO:0140098']
 ['GO:0003824' 'GO:0009975']
 ['GO:0003824' 'GO:0016829']
 ['GO:0003824' 'GO:0016740']
 ['GO:0003824' 'GO:0140097']
 ['GO:0016787' 'GO:0003924']
 ['GO:0098772' 'GO:0048018']
 ['GO:0003674' 'GO:0098772']
 ['GO:0003674' 'GO:0045182']
 ['GO:0003674' 'GO:0140657']
 ['GO:0003674' 'GO:0003824']
 ['GO:0003674' 'GO:0044183']
 ['GO:0003674' 'GO:0031386']
 ['GO:0003674' 'GO:0048018']
 ['GO:0003674' 'GO:0016853']
 ['GO:0003674' 'GO:0090729']
 ['GO:0003674' 'GO:0016209']
 ['GO:0003674' 'GO:0005198']
 ['GO:0003674' 'GO:0060090']
 ['GO:0003674' 'GO:0016491']
 ['GO:0003674'

sort_pairs 测试

In [84]:

def sort_pairs(genes_terms_pairs, terms_pairs, dG, gene2id_mapping):
    """
    Function concatenates the pairs and orders them, the parent term goes first.

        Output
        ------
        level_list: list
            Each array of the list stores the elements on a level of the hierarchy

        level_number: dict
            Has the gene and GO terms with their corresponding level number

        sorted_pairs: numpy.ndarray
            Contains the term-gene or term-term pairs with the parent element on the first column
    """

    all_pairs = np.concatenate((genes_terms_pairs,terms_pairs))  # 把GO-gene和GO-GO的两个numpy数组连接起来
    graph = dG.copy() #  Copy the graph to avoid modifying the original

    level_list = []   # level_list stores the elements on each level of the hierarchy
    level_list.append(list(gene2id_mapping.keys())) # add the genes 先把所有基因存到List中

    while True:
        # 找到这一层的所有GO  一开始找到的是最底层的直接和gene相关的GO 从下往上编层数号 
        leaves = [n for n in graph.nodes() if graph.out_degree(n) == 0]

        if len(leaves) == 0:
            break
        # 依次从叶子节点GO开始把GO存到level_list中 按列表形式存储 表示每一层级的GO
        level_list.append(leaves) # add the terms on each level
        graph.remove_nodes_from(leaves)

    # 给每个GO和gene打一个层级标签 表示是GO图中的第几层级
    # level_number应该包含所有GO和gene的一个字典 每个Key对应一个值 表示这个GO/gene的层级
    # 第一层是gene 因为循环前先把gene存到level_list中了 第二层开始才是与gene直接相连的GO
    level_number = {} # Has the gene and GO terms with their corresponding level number
    for i, layer in enumerate(level_list):  # layer是这一层的GO列表/gene列表
        for _,item in enumerate(layer):  # 遍历这一层的GO/gene列表 item是这一层每个GO/gene
            level_number[item] = i

    # 按照层级 对所有的pair进行排序 使得层级低的GO在后面 即高级别的父节点GO在前面 低级别的子节点GO在后面
    # 这里的层级和一开始文件中天然的父子关系不同 因为建好的GO图中 不同GO的层级所在不一定和父子关系一致
    sorted_pairs = all_pairs.copy() # order pairs based on their level
    for i, pair in enumerate(sorted_pairs):
        level1 = level_number[pair[0]]
        level2 = level_number[pair[1]]
        if level2 > level1:  # the parent term goes first
            sorted_pairs[i][1] = all_pairs[i][0]
            sorted_pairs[i][0] = all_pairs[i][1]
    # 返回排好序的pair 按等级排好序的GO level_list 以及GO和gene的层级标签字典
    return sorted_pairs, level_list, level_number

sorted_pairs, level_list, level_number = sort_pairs(
    genes_terms_pairs, terms_pairs, dG, gene2id_mapping)
print(sorted_pairs[1])
print(level_list[4])
for i in level_list:
    print(len(i))
print(level_number)

['GO:0003824' 'A100048']
['GO:0003674', 'GO:0005575']
3100
125
13
3
2
1
{'A100048': 0, 'A100051': 0, 'A100062': 0, 'A100211': 0, 'A100223': 0, 'A100230': 0, 'A100246': 0, 'A100384': 0, 'A100496': 0, 'A100598': 0, 'A100633': 0, 'A100689': 0, 'A100702': 0, 'A101024': 0, 'A101032': 0, 'A101033': 0, 'A101046': 0, 'A101069': 0, 'A101115': 0, 'A101132': 0, 'A101348': 0, 'A101355': 0, 'A101362': 0, 'A101496': 0, 'A101775': 0, 'A101906': 0, 'A102725': 0, 'A103111': 0, 'A103121': 0, 'A103157': 0, 'A103343': 0, 'A103523': 0, 'A103549': 0, 'A103624': 0, 'A103685': 0, 'A104785': 0, 'A104969': 0, 'A105420': 0, 'A105804': 0, 'A107159': 0, 'A107816': 0, 'A108301': 0, 'A108590': 0, 'A111449': 0, 'A112027': 0, 'A112030': 0, 'A112046': 0, 'A112047': 0, 'A112051': 0, 'A112062': 0, 'A112076': 0, 'A112099': 0, 'A112100': 0, 'A112127': 0, 'A112141': 0, 'A112147': 0, 'A112153': 0, 'A112157': 0, 'A112161': 0, 'A112169': 0, 'A112170': 0, 'A112172': 0, 'A112178': 0, 'A112179': 0, 'A112185': 0, 'A112216': 0, 'A1

pairs_in_layers 加入虚拟节点 测试

In [95]:

def pairs_in_layers(sorted_pairs, level_list, level_number):
    """
    This function divides all the pairs of GO terms and genes by layers and adds the virtual nodes

        Output
        ------
        layer_connections: numpy.ndarray
            Contains the pairs that will be part of each layer of the model.
            Not all terms are connected to a term on the level above it. "Virtual nodes" are added to establish the connections between non-subsequent levels.

    """
    total_layers = len(level_list)-1 # Number of layers that the model will contain 总层级个数
    # Will contain the GO terms connections and gene-term connections by layers
    layer_connections = [[] for i in range(total_layers)]  # 先给每一层创建一个空列表

    # 遍历按照层级排序好的pair
    for i, pair in enumerate(sorted_pairs):
        parent = level_number[pair[0]]  #  父节点的层级 这里是层级更高的GO的层级
        child = level_number[pair[1]]  # 子节点的层级 这里是层级更低的GO/gene的层级
        # 注意这里的一个骚操作： 因为都是在child所在层添加pair 所以会遍历到gene时 每个gene相关的GO都会添加到第一层里
        # 也就是说 无论gene相关的GO究竟在哪一层 都会被一路虚拟节点拉回第一层
        # 这就保证了建立的GO网络中 第一层的维度一定是所有GO个数的维度 (因为已确保每个GO至少和一个gene有直接关系)
        # 上面这个假设好像有点问题 如果有一个GO和所有gene都没有直接关系 但这个GO的子节点有关系 好像也成立
        # 这样子第一层输入就不一定是所有GO的长度了
        # 这就确定了网络的输入维度  关于输出维度好像一定是1 可能是回归问题设置的 我觉着我不需要这个限制

        # Add the pair to its corresponding layer
        layer_connections[child].append(pair)  # 找到child对应的层级 把这一对pair添加到这一层中

        # If the pair is not directly connected virtual nodes have to be added
        # 如果不是相邻层之间相连 则需要添加虚拟节点
        # 我们不考虑蛋白质与GO之间的关系 直接把GO网络所有GO都拉下来
        if child != 0:  # 如果child不是蛋白质 则进行GO虚拟节点调整
            virtual_node_layer = parent - 1  # 初始补充层为 parent-1 (最高层下一层)
            while virtual_node_layer >= 1:  # 忽略第一层 gene，从第二层开始
            # for j in range(dif-1): # 循环需要补充的差值层次数 依次修改virtual_node_layer 修改补充层位置
                layer_connections[virtual_node_layer].append([pair[0],pair[0]])  # 循环地在这个中间层中添加虚拟节点(父节点-父节点 表示虚拟节点)
                virtual_node_layer = virtual_node_layer-1

    # Delete pairs that are duplicated (added twice on the above step)
    # 删除重复的对 有可能在上面的循环中添加了两次
    for i,_ in enumerate(layer_connections):
        layer_connections[i] = np.array(layer_connections[i]) # change list to array
        layer_connections[i] = np.unique(layer_connections[i], axis=0)

    return layer_connections

layer_connections = pairs_in_layers(sorted_pairs, level_list, level_number)  # 添加虚拟节点
print(len(layer_connections[0]))
print(len(layer_connections[1]))
# print(layer_connections[1])
print(len(layer_connections[2]))
print(len(layer_connections[3]))
# print(len(layer_connections[4]))
# print(len(layer_connections[5]))
layer_nodes = []
# 第二层就是GO最初始层 里面一共涉及了144个GO 全部GO  
# layer_connections存的是GO-GO之间的关系 里面是若干个pairs 不是按GO network的节点顺序存的
for i, j in enumerate(layer_connections[1]):
    layer_nodes.append(j[1])
layer_nodes = list(set(layer_nodes))
print(len(layer_nodes))



18358
187
25
6
144


后来发现GO net搭建的时候会根据逻辑固定input顺序
这里按照layer_connection排列的index也是不准确的 所以不用再做新的input
input就按onehot/multi_hot输入即可
因为第一层index已经人为写死在模型搭建中
人为对齐了两个输入
onehot生成时的Input顺序和模型里的Input顺序是一致的

根据GO net第二层id的顺序建立 protein input 这里先建立protein2GO.pkl

In [ ]:

import pandas as pd
import pickle

def create_index(array):
    unique_array = pd.unique(array)

    index = {}
    for i, element in enumerate(unique_array):
        index[element] = i

    return index
print(len(layer_connections[0][:,0]))
# 记录网络第一层的GO id顺序
input_id = create_index(layer_connections[0][:,0])
print(len(input_id))
# for i, go_id in enumerate(input_id):
#     print(f"Neuron {i} corresponds to GO id {go_id}")
#     # break

# 建立一个同等长度的vector
vector = [0] * len(input_id)

# 读取txt文件
df = pd.read_csv('./data/noise_free/eSOL_go/eSOL_test_total.txt', sep=' ', header=None)

# 找到第三列是protein的所有行
protein_rows = df[df[2] == 'protein']

# 整理成protein:{GO id}的形式
protein_dict = protein_rows.groupby(1)[0].apply(list).to_dict()

# 遍历每个protein
for protein, go_ids in protein_dict.items():
    # 每个protein建立一个字典
    protein_dict[protein] = dict.fromkeys(input_id, 0)
    # 对每个GO id填充一个值作为value
    for go_id in go_ids:
        if go_id in protein_dict[protein]:
            protein_dict[protein][go_id] = 1

# 保存成pkl文件
with open('./data/noise_free/eSOL_go/eSOL_test_protein2GO_new.pkl', 'wb') as f:
    pickle.dump(protein_dict, f)

18358
136


然后建立 GO input.pkl

In [ ]:
import pickle
import numpy as np

# 读取pkl文件
with open('./data/noise_free/eSOL_go/eSOL_test_protein2GO_new.pkl', 'rb') as f:
    protein_dict = pickle.load(f)

# 新建一个字典，用于保存protein id和对应的vector
protein_vector_dict = {}

# 遍历每个protein
for protein, go_dict in protein_dict.items():
    # 按照input_id的顺序给vector填充值
    vector = [0] * len(input_id)
    for i, go_id in enumerate(input_id):
        if go_dict[go_id] == 1:
            vector[i] = 1
    # 将protein id和对应的vector添加到新的字典中
    protein_vector_dict[protein] = np.array(vector)



# 保存成pkl文件
with open('./data/noise_free/eSOL_go/eSOL_test_go_onehot_input_new.pkl', 'wb') as f:
    pickle.dump(protein_vector_dict, f)